In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np

In [2]:
image_train = pd.read_csv(r"image_train_data.csv")
image_test = pd.read_csv(r"image_test_data.csv")

In [3]:
len(image_train)

2005

In [4]:
image_train.head()

,id,image,label,deep_features,image_array
0,24,Height: 32 Width: 32,bird,[0.242872 1.09545 0 0.39363 0 0 11.8949 0 0 0 ...,[73 77 58 71 68 50 77 69 44 120 116 83 125 120...
1,33,Height: 32 Width: 32,cat,[0.525088 0 0 0 0 0 9.94829 0 0 0 0 0 1.01264 ...,[7 5 8 7 5 8 5 4 6 7 4 7 11 5 9 11 5 9 17 11 1...
2,36,Height: 32 Width: 32,cat,[0.566016 0 0 0 0 0 9.9972 0 0 0 1.38345 0 0.7...,[169 122 65 131 108 75 193 196 192 218 221 222...
3,70,Height: 32 Width: 32,dog,[1.1298 0 0 0.778194 0 0.758051 9.83053 0 0 0....,[154 179 152 159 183 157 165 189 162 174 199 1...
4,90,Height: 32 Width: 32,bird,[1.71787 0 0 0 0 0 9.33936 0 0 0 0 0 0.412137 ...,[216 195 180 201 178 160 210 184 164 212 188 1...


In [5]:
image_train['image_array'][0]

'[73 77 58 71 68 50 77 69 44 120 116 83 125 120 90 155 148 117 147 152 106 133 136 92 112 101 75 100 84 68 100 78 74 69 49 51 22 16 13 35 27 24 52 40 40 70 55 54 117 97 89 122 101 99 103 85 88 88 74 73 68 59 52 72 65 52 82 73 60 84 72 68 84 67 73 77 56 65 93 75 81 117 102 98 174 171 146 183 210 149 180 214 143 185 225 144 73 74 52 75 68 55 78 64 50 103 90 64 80 65 35 127 110 80 136 127 98 141 134 109 129 113 92 115 97 72 111 91 61 115 96 67 94 83 65 92 78 65 78 60 53 89 68 64 123 99 92 117 93 92 118 97 102 121 103 103 137 125 116 168 159 141 134 124 104 98 86 74 135 124 115 133 131 108 158 167 126 163 185 125 184 217 143 181 229 139 172 220 129 174 218 127 77 72 64 94 82 79 99 81 73 116 97 74 130 106 74 149 122 93 138 118 101 134 121 109 148 140 117 154 143 113 149 137 99 145 127 87 168 136 102 174 137 111 180 137 121 178 134 126 160 126 127 141 112 124 110 84 100 110 87 99 105 90 94 124 121 106 151 162 119 155 177 117 190 216 152 192 220 151 201 236 156 193 234 155 165 208 135 175 213

In [6]:
type(image_train['image_array'][0])

str

In [7]:
check = image_train['image_array'][0]
print(check[0])      #First element of the string is a paranthesis
print(check[1])      #Second element of the string is the first digit of the first number
print(check[2])      #Third element of the string is the second digit of the first number
print(check[3])      #Fourth element of the string is a space

[
7
3
 


# Function for converting the input data of strings into list of floats.

In [8]:
def extract_features(x):
    raw = re.search(r"\[(.*)\]", x).group(1)
    raw = list(raw.split())
    raw = list(map(float, raw))
    raw = (np.array(raw)).reshape(1,-1)
    return raw

# Fixing the train data set

## Converting 'image_array' column to a meaningful feature

In [9]:
image_train.image_array = image_train.image_array.apply(lambda x: extract_features(x))

## Converting 'deep_features' column to a meaningful feature

In [10]:
image_train.deep_features = image_train.deep_features.apply(lambda x: extract_features(x))

# Fixing the train data set

## Converting 'image_array' column to a meaningful feature

In [11]:
image_test.image_array = image_test.image_array.apply(lambda x: extract_features(x))

## Converting 'deep_features' column to a meaningful feature

In [12]:
image_test.deep_features = image_test.deep_features.apply(lambda x: extract_features(x))

# Train an image classifier on raw image pixels('image_array' column)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

x = np.concatenate(image_train['image_array'].values)
y = image_train['label']

raw_pixel_model = KNeighborsClassifier(n_neighbors=5)
raw_pixel_model.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [14]:
image_test[0:3]['label']

0           cat
1    automobile
2           cat
Name: label, dtype: object

In [15]:
distance, indices = raw_pixel_model.kneighbors(np.concatenate(image_test['image_array'][0:1]), return_distance=True, n_neighbors = 1)
image_train.iloc[indices[0], [2]]

,label
1391,bird


In [16]:
raw_pixel_model.kneighbors(np.concatenate(image_test['image_array'][0:1]), return_distance=True, n_neighbors = 5)

(array([[2712.45866328, 2731.67018507, 2765.02206139, 2815.69440813,
         2825.84624493]]),
 array([[1391, 1672, 1087, 1610,  156]], dtype=int64))

# Function to get the 5 nearest neighbours

In [17]:
def five_neighbours(idx):
    distance, indices = raw_pixel_model.kneighbors(np.concatenate(image_test['image_array'][idx]), n_neighbors = 5)
    index_distance_df = image_train.iloc[indices[0], [2]]
    index_distance_df['distance'] = distance[0]
    return index_distance_df

# Applying the raw_pixel_model to predict the nearest neighbours of 1st row of the test data with the train data

In [18]:
image_test[0:1]['label']

0    cat
Name: label, dtype: object

In [19]:
cat_test = image_test.iloc[0:1, 2].index.values.astype(int)

In [20]:
five_neighbours(cat_test)

,label,distance
1391,bird,2712.458663
1672,bird,2731.670185
1087,bird,2765.022061
1610,bird,2815.694408
156,dog,2825.846245


# Train image classifier using deep features (Transfer learning using imagenet learned features)

In [21]:
x_d = np.concatenate(image_train['deep_features'].values)
y_d = image_train['label']

deep_features_model = KNeighborsClassifier(n_neighbors=5)
deep_features_model.fit(x_d, y_d)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

# Function to get the 5 nearest neighbours

In [22]:
def five_neighbours_deep(idx):
    distance, indices = deep_features_model.kneighbors(np.concatenate(image_test['deep_features'][idx]), n_neighbors = 5)
    index_distance_df = image_train.iloc[indices[0], [2]]
    index_distance_df['distance'] = distance[0]
    return index_distance_df

# Applying the deep_features_model to predict the nearest neighbours of 1st row of the test data with the train data

In [23]:
five_neighbours_deep(cat_test)

,label,distance
669,cat,34.623722
1815,cat,36.006880
1299,cat,36.520081
1064,cat,36.754848
15,cat,36.873115


In [24]:
x_test_deep_feature = np.concatenate(image_test['deep_features'].values)
x_test_raw_pixel = np.concatenate(image_test['image_array'].values)
y_test = image_test['label']

In [25]:
y_pred_raw_pixel = raw_pixel_model.predict(x_test_raw_pixel)
y_pred_deep_feature = deep_features_model.predict(x_test_deep_feature)

In [26]:
from sklearn import metrics

print("The accuracy in predicting the test data label for raw pixel model is : ", metrics.accuracy_score(y_test, y_pred_raw_pixel))
print("The accuracy in predicting the test data label for deep feature model is : ", metrics.accuracy_score(y_test, y_pred_deep_feature))

The accuracy in predicting the test data label for raw pixel model is :  0.391
The accuracy in predicting the test data label for deep feature model is :  0.74475


# Check individual count of unique labels in train and test sets

In [27]:
image_train.groupby(['label']).size().reset_index(name='counts')

,label,counts
0,automobile,509
1,bird,478
2,cat,509
3,dog,509


In [28]:
image_test.groupby(['label']).size().reset_index(name='counts')

,label,counts
0,automobile,1000
1,bird,1000
2,cat,1000
3,dog,1000


# Train seperate models for unique labels using train images

In [29]:
cat_train = image_train[image_train['label'] == 'cat']
dog_train = image_train[image_train['label'] == 'dog']
automobile_train = image_train[image_train['label'] == 'automobile']
bird_train = image_train[image_train['label'] == 'bird']

# Create a nearest neighbor model using deep features for each of the four labels (Only deep_features used)

In [30]:
x_cat = np.concatenate(cat_train['deep_features'].values)
y_cat = cat_train['label']

cat_model = KNeighborsClassifier(n_neighbors=5)
cat_model.fit(x_cat, y_cat)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [31]:
x_dog = np.concatenate(dog_train['deep_features'].values)
y_dog = dog_train['label']

dog_model = KNeighborsClassifier(n_neighbors=5)
dog_model.fit(x_dog, y_dog)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [32]:
x_automobile = np.concatenate(automobile_train['deep_features'].values)
y_automobile = automobile_train['label']

automobile_model = KNeighborsClassifier(n_neighbors=5)
automobile_model.fit(x_automobile, y_automobile)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [33]:
x_bird = np.concatenate(bird_train['deep_features'].values)
y_bird = bird_train['label']

bird_model = KNeighborsClassifier(n_neighbors=5)
bird_model.fit(x_bird, y_bird)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

# Function to get the 5 nearest neighbours from different models when input feature is given as a query

In [34]:
cat_test_1 = cat_train.iloc[0:1, 2].index.values.astype(int)
cat_test_1[0]

1

In [35]:
def five_neighbours_cat(idx):
    distance, indices = cat_model.kneighbors(np.concatenate(cat_train['deep_features'][idx[0]:idx[0]+1].values), n_neighbors = 5)
    index_distance_df = cat_train.iloc[indices[0], [2]]
    index_distance_df['distance'] = distance[0]
    return index_distance_df

In [36]:
five_neighbours_cat(cat_test_1)

,label,distance
2,cat,0.000000
1639,cat,30.736229
1552,cat,31.087095
448,cat,31.446784
773,cat,31.885011


In [37]:
print("The mean distance between the cat in test data with the 5 nearest neighbours in the cat_train is : ",five_neighbours_cat(cat_test)['distance'].mean())

The mean distance between the cat in test data with the 5 nearest neighbours in the cat_train is :  27.877732467312462


In [38]:
def five_neighbours_dog(idx):
    distance, indices = dog_model.kneighbors(np.concatenate(cat_train['deep_features'][idx[0]:idx[0]+1].values), n_neighbors = 5)
    index_distance_df = dog_train.iloc[indices[0], [2]]
    index_distance_df['distance'] = distance[0]
    return index_distance_df

In [39]:
five_neighbours_dog(cat_test_1)

,label,distance
1689,dog,33.557857
678,dog,34.315702
1212,dog,35.493708
1270,dog,35.779065
683,dog,36.160117


In [40]:
print("The mean distance between the cat in test data with the 5 nearest neighbours in the dog_train is : ",five_neighbours_dog(cat_test)['distance'].mean())

The mean distance between the cat in test data with the 5 nearest neighbours in the dog_train is :  32.77346949525377


In [41]:
cat_train['deep_features'][1:4].values

array([array([[0.566016, 0.      , 0.      , ..., 0.      , 0.      , 0.      ]]),
       array([[0.      , 0.      , 0.      , ..., 0.481036, 0.      , 0.      ]]),
       array([[0.      , 0.      , 0.510964, ..., 0.      , 0.      , 0.      ]])],
      dtype=object)

# Query the whole test set instead of one single row and test for 1 nearest neighbour for the whole test data against different models

In [42]:
cat_test = image_test[image_test['label'] == 'cat']

In [43]:
dog_test = image_test[image_test['label'] == 'dog']

In [44]:
automobile_test = image_test[image_test['label'] == 'automobile']

In [45]:
bird_test = image_test[image_test['label'] == 'bird']

# Functions to get nearest dog-"x" pairs using dog_test as the input query and getting 1 nearest neighbour for each dog label against all the 4 different models

# Find the nearest neighbor for each dog in dog_test data with cat_train using cat model

In [176]:
label_dog_cat = []
distance_dog_cat = []

def dog_cat_neighbours(x):
    distance, indices = cat_model.kneighbors(x, n_neighbors = 1)
    label_dog_cat.append(indices)
    distance_dog_cat.append(distance)
    return 0

In [177]:
return_1 = dog_test['deep_features'].apply(lambda x: dog_cat_neighbours(x))

In [178]:
df_dog_cat = pd.DataFrame({'distance' : distance_dog_cat, 'label' : label_dog_cat})
df_dog_cat

,distance,label
0,[[36.41960343224118]],[[0]]
1,[[38.83532565622547]],[[308]]
2,[[36.97633927904312]],[[58]]
3,[[34.575005304480044]],[[224]]
4,[[34.77882715084174]],[[84]]
...,...,...
995,[[43.477972886076124]],[[224]]
996,[[34.326576839006336]],[[396]]
997,[[35.98349831913211]],[[340]]
998,[[30.517842455591797]],[[32]]


In [179]:
def convert_array(y):
    return np.sum(y)

In [180]:
df_dog_cat['distance'] = df_dog_cat['distance'].apply(lambda y: convert_array(y))
df_dog_cat['label'] = df_dog_cat['label'].apply(lambda y: convert_array(y))

In [181]:
df_dog_cat

,distance,label
0,36.419603,0
1,38.835326,308
2,36.976339,58
3,34.575005,224
4,34.778827,84
...,...,...
995,43.477973,224
996,34.326577,396
997,35.983498,340
998,30.517842,32


# Find the nearest neighbor for each dog in dog_test data with dog_train using dog model

In [154]:
label_dog_dog = []
distance_dog_dog = []

def dog_dog_neighbours(x):
    distance, indices = dog_model.kneighbors(x, n_neighbors = 1)
    label_dog_dog.append(indices)
    distance_dog_dog.append(distance)
    return 0

In [155]:
return_2 = dog_test['deep_features'].apply(lambda x: dog_dog_neighbours(x))

In [156]:
df_dog_dog = pd.DataFrame({'distance' : distance_dog_dog, 'label' : label_dog_dog})

In [157]:
df_dog_dog['distance'] = df_dog_dog['distance'].apply(lambda y: convert_array(y))
df_dog_dog['label'] = df_dog_dog['label'].apply(lambda y: convert_array(y))

In [158]:
df_dog_dog

,distance,label
0,33.477354,506
1,32.845849,62
2,35.039707,203
3,33.901032,129
4,37.484928,120
...,...,...
995,38.874712,54
996,35.653573,139
997,36.515231,22
998,32.528297,293


# Find the nearest neighbor for each dog in dog_test data with automobile_train using automobile model

In [159]:
label_dog_automobile = []
distance_dog_automobile = []

def dog_automobile_neighbours(x):
    distance, indices = automobile_model.kneighbors(x, n_neighbors = 1)
    label_dog_automobile.append(indices)
    distance_dog_automobile.append(distance)
    return 0

In [161]:
return_3 = dog_test['deep_features'].apply(lambda x: dog_automobile_neighbours(x))

In [162]:
df_dog_automobile = pd.DataFrame({'distance' : distance_dog_automobile, 'label' : label_dog_automobile})

In [163]:
df_dog_automobile['distance'] = df_dog_automobile['distance'].apply(lambda y: convert_array(y))
df_dog_automobile['label'] = df_dog_automobile['label'].apply(lambda y: convert_array(y))

In [164]:
df_dog_automobile

,distance,label
0,41.957982,350
1,46.002135,17
2,42.946231,191
3,41.686604,108
4,39.226973,191
...,...,...
995,44.309540,401
996,44.519197,205
997,42.326395,371
998,39.457429,191


# Find the nearest neighbor for each dog in dog_test data with bird_train using bird model

In [165]:
label_dog_bird = []
distance_dog_bird = []

def dog_bird_neighbours(x):
    distance, indices = bird_model.kneighbors(x, n_neighbors = 1)
    label_dog_bird.append(indices)
    distance_dog_bird.append(distance)
    return 0

In [166]:
return_4 = dog_test['deep_features'].apply(lambda x: dog_bird_neighbours(x))

In [167]:
df_dog_bird = pd.DataFrame({'distance' : distance_dog_bird, 'label' : label_dog_bird})

In [168]:
df_dog_bird['distance'] = df_dog_bird['distance'].apply(lambda y: convert_array(y))
df_dog_bird['label'] = df_dog_bird['label'].apply(lambda y: convert_array(y))

In [169]:
df_dog_bird

,distance,label
0,41.753863,420
1,41.338292,94
2,38.615761,353
3,37.089222,132
4,38.272291,347
...,...,...
995,40.397742,211
996,38.816914,71
997,37.722023,347
998,37.264525,40


# Check for the total number of rows where the distance of dog_dog is less than other distances

In [182]:
dog_distances = pd.DataFrame({'dog-dog': df_dog_dog['distance'],'dog-cat': df_dog_cat['distance'], 'dog-automobile': df_dog_automobile['distance'], 'dog-bird': df_dog_bird['distance']})

In [184]:
dog_distances['row_minimum'] = dog_distances.min(axis=1)

In [185]:
dog_distances

,dog-dog,dog-cat,dog-automobile,dog-bird,row_minimum
0,33.477354,36.419603,41.957982,41.753863,33.477354
1,32.845849,38.835326,46.002135,41.338292,32.845849
2,35.039707,36.976339,42.946231,38.615761,35.039707
3,33.901032,34.575005,41.686604,37.089222,33.901032
4,37.484928,34.778827,39.226973,38.272291,34.778827
...,...,...,...,...,...
995,38.874712,43.477973,44.309540,40.397742,38.874712
996,35.653573,34.326577,44.519197,38.816914,34.326577
997,36.515231,35.983498,42.326395,37.722023,35.983498
998,32.528297,30.517842,39.457429,37.264525,30.517842


In [186]:
def is_dog_correct(row):
    if row['dog-dog'] <= row['row_minimum']:
        return 1
    return 0

In [188]:
total_correct_dog_predictions = dog_distances.apply(is_dog_correct, axis=1).sum()
total_correct_dog_predictions

678

In [189]:
accuracy_dog_predictions = total_correct_dog_predictions/len(dog_distances)
accuracy_dog_predictions

0.678